In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import zscore

Data important features:
position - (Integer) - Hotel position on Expedia's search results page. This is only provided for the training data, but not the test data.
click_bool - (Boolean) - 1 if the user clicked on the property, 0 if not.
booking_bool - (Boolean) - 1 if the user booked the property, 0 if not.
gross_booking_usd - (Float) - Total value of the transaction. This can differ from the price_usd due to taxes, fees, conventions on multiple day bookings and purchase of a room type other than the one shown in the search.

More info on: https://www.kaggle.com/c/expedia-personalized-sort/data

In [23]:
test_data = pd.read_csv('data/test.csv')
train_data = pd.read_csv('data/train.csv')

print(train_data.head())

   srch_id            date_time  site_id  visitor_location_country_id  \
0        1  2013-04-04 08:32:15       12                          187   
1        1  2013-04-04 08:32:15       12                          187   
2        1  2013-04-04 08:32:15       12                          187   
3        1  2013-04-04 08:32:15       12                          187   
4        1  2013-04-04 08:32:15       12                          187   

   visitor_hist_starrating  visitor_hist_adr_usd  prop_country_id  prop_id  \
0                      NaN                   NaN              219      893   
1                      NaN                   NaN              219    10404   
2                      NaN                   NaN              219    21315   
3                      NaN                   NaN              219    27348   
4                      NaN                   NaN              219    29604   

   prop_starrating  prop_review_score  ...  comp6_rate_percent_diff  \
0                3   

In [24]:
# print size of data
print(train_data.shape)

# display data information
train_data.info()

(4958347, 54)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4958347 entries, 0 to 4958346
Data columns (total 54 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   srch_id                      int64  
 1   date_time                    object 
 2   site_id                      int64  
 3   visitor_location_country_id  int64  
 4   visitor_hist_starrating      float64
 5   visitor_hist_adr_usd         float64
 6   prop_country_id              int64  
 7   prop_id                      int64  
 8   prop_starrating              int64  
 9   prop_review_score            float64
 10  prop_brand_bool              int64  
 11  prop_location_score1         float64
 12  prop_location_score2         float64
 13  prop_log_historical_price    float64
 14  position                     int64  
 15  price_usd                    float64
 16  promotion_flag               int64  
 17  srch_destination_id          int64  
 18  srch_length_of_stay         

Non ordinal integers that should become strings:
- srch_destination_id
- site_id 
- visitor_location_country_id
- prop_country_id
- srch_destination_id 

Integers that should become boolean:
- prop_brand_bool
- srch_saturday_night_bool
- random_bool

To predict:
position: Hotel position on Expedia's search results page. This is only provided for the training data, but not the test data.

In [25]:
# transform integers to strings for categorical data
train_data['srch_destination_id'] = train_data['srch_destination_id'].astype(str)
train_data['site_id'] = train_data['site_id'].astype(str)
train_data['visitor_location_country_id'] = train_data['visitor_location_country_id'].astype(str)
train_data['prop_country_id'] = train_data['prop_country_id'].astype(str)
train_data['prop_id'] = train_data['prop_id'].astype(str)
train_data['srch_destination_id'] = train_data['srch_destination_id'].astype(str)

In [26]:
# transform integers to boleans
train_data["prop_brand_bool"] = train_data["prop_brand_bool"].astype(bool)
train_data["srch_saturday_night_bool"] = train_data["srch_saturday_night_bool"].astype(bool)
train_data["random_bool"] = train_data["random_bool"].astype(bool)
train_data["promotion_flag"] = train_data["promotion_flag"].astype(bool)

In [27]:
# print missing values in each column
for col in train_data.columns:
    # print percentage of missing values
    print(col, train_data[col].isnull().sum()/train_data.shape[0])

srch_id 0.0
date_time 0.0
site_id 0.0
visitor_location_country_id 0.0
visitor_hist_starrating 0.949203635808466
visitor_hist_adr_usd 0.9489773507178905
prop_country_id 0.0
prop_id 0.0
prop_starrating 0.0
prop_review_score 0.0014851723770038683
prop_brand_bool 0.0
prop_location_score1 0.0
prop_location_score2 0.2199015115319682
prop_log_historical_price 0.0
position 0.0
price_usd 0.0
promotion_flag 0.0
srch_destination_id 0.0
srch_length_of_stay 0.0
srch_booking_window 0.0
srch_adults_count 0.0
srch_children_count 0.0
srch_room_count 0.0
srch_saturday_night_bool 0.0
srch_query_affinity_score 0.935985520981085
orig_destination_distance 0.32425766086964064
random_bool 0.0
comp1_rate 0.9758125036428471
comp1_inv 0.9738705258022482
comp1_rate_percent_diff 0.9809535314894258
comp2_rate 0.5916639154137457
comp2_inv 0.5703671001646314
comp2_rate_percent_diff 0.8878178554264153
comp3_rate 0.6905646176034069
comp3_inv 0.6670281446619206
comp3_rate_percent_diff 0.9046462460170698
comp4_rate 0.938

In [28]:
# print all columns that contain more than 50% missing values
for col in train_data.columns:
    # print percentage of missing values
    if train_data[col].isnull().sum()/train_data.shape[0] > 0.5:
        print(col, train_data[col].isnull().sum()/train_data.shape[0])

visitor_hist_starrating 0.949203635808466
visitor_hist_adr_usd 0.9489773507178905
srch_query_affinity_score 0.935985520981085
comp1_rate 0.9758125036428471
comp1_inv 0.9738705258022482
comp1_rate_percent_diff 0.9809535314894258
comp2_rate 0.5916639154137457
comp2_inv 0.5703671001646314
comp2_rate_percent_diff 0.8878178554264153
comp3_rate 0.6905646176034069
comp3_inv 0.6670281446619206
comp3_rate_percent_diff 0.9046462460170698
comp4_rate 0.9380079691881186
comp4_inv 0.9306900061653611
comp4_rate_percent_diff 0.9735625602645398
comp5_rate 0.5517915547257988
comp5_inv 0.5240308917467857
comp5_rate_percent_diff 0.8303670557950059
comp6_rate 0.9515651082911301
comp6_inv 0.9473663299482671
comp6_rate_percent_diff 0.9806036164875108
comp7_rate 0.9364005786605899
comp7_inv 0.9281167695605007
comp7_rate_percent_diff 0.9720642786799714
comp8_rate 0.61344899822461
comp8_inv 0.5991601636593809
comp8_rate_percent_diff 0.8760211820592629
gross_bookings_usd 0.9720894886945186


In [29]:
# transform all integers to floats to compute means and z-scores
for col in train_data.columns:
    if train_data[col].dtype == 'int64':
        train_data[col] = train_data[col].astype(float)        

In [30]:
# create a table for all numerical data with mean, median, highest, lowest and standard deviation
num_describtion = train_data.describe()
print(num_describtion)

# save to latex table
import os
if not os.path.exists('latex'):
    os.makedirs('latex')
num_describtion.to_latex('latex/num_describtion.tex')

            srch_id  visitor_hist_starrating  visitor_hist_adr_usd  \
count  4.958347e+06            251866.000000         252988.000000   
mean   1.663666e+05                 3.374334            176.022659   
std    9.611223e+04                 0.692519            107.254493   
min    1.000000e+00                 1.410000              0.000000   
25%    8.293600e+04                 2.920000            109.810000   
50%    1.665070e+05                 3.450000            152.240000   
75%    2.497240e+05                 3.930000            213.490000   
max    3.327850e+05                 5.000000           1958.700000   

       prop_starrating  prop_review_score  prop_location_score1  \
count     4.958347e+06       4.950983e+06          4.958347e+06   
mean      3.180525e+00       3.777777e+00          2.872589e+00   
std       1.051024e+00       1.050329e+00          1.531011e+00   
min       0.000000e+00       0.000000e+00          0.000000e+00   
25%       3.000000e+00       3.500

In [31]:
# get number of unique categories, most occuring, least occuring and percentage of most occuring category for categorical data
dict_cat = {}
for col in train_data.columns:
    if train_data[col].dtype == 'object':
        unique = train_data[col].nunique()
        moc = train_data[col].value_counts().idxmax()
        moc_freq = train_data[col].value_counts().max()/train_data.shape[0]
        loc = train_data[col].value_counts().idxmin()
        loc_freq = train_data[col].value_counts().min()/train_data.shape[0]
        
        dict_cat[col] = [unique, moc, moc_freq, loc, loc_freq]
    
# create table for categorical data
cat_describtion = pd.DataFrame.from_dict(dict_cat, orient='index', columns=['unique', 'most occuring', 'most occuring frequency', 'least occuring', 'least occuring frequency'])

# save to latex
cat_describtion.to_latex('cat_describtion.tex')

print(cat_describtion)

                             unique        most occuring  \
date_time                    198615  2012-11-02 14:04:51   
site_id                          34                    5   
visitor_location_country_id     210                  219   
prop_country_id                 172                  219   
prop_id                      129113               104517   
srch_destination_id           18127                 8192   

                             most occuring frequency       least occuring  \
date_time                                   0.000019  2013-02-07 11:25:00   
site_id                                     0.622386                    8   
visitor_location_country_id                 0.583358                  150   
prop_country_id                             0.611116                  165   
prop_id                                     0.000475               108653   
srch_destination_id                         0.014084                27364   

                             least occu

In [32]:
# print all column names and their data type
for col in train_data.columns:
    print(col, train_data[col].dtype)

srch_id float64
date_time object
site_id object
visitor_location_country_id object
visitor_hist_starrating float64
visitor_hist_adr_usd float64
prop_country_id object
prop_id object
prop_starrating float64
prop_review_score float64
prop_brand_bool bool
prop_location_score1 float64
prop_location_score2 float64
prop_log_historical_price float64
position float64
price_usd float64
promotion_flag bool
srch_destination_id object
srch_length_of_stay float64
srch_booking_window float64
srch_adults_count float64
srch_children_count float64
srch_room_count float64
srch_saturday_night_bool bool
srch_query_affinity_score float64
orig_destination_distance float64
random_bool bool
comp1_rate float64
comp1_inv float64
comp1_rate_percent_diff float64
comp2_rate float64
comp2_inv float64
comp2_rate_percent_diff float64
comp3_rate float64
comp3_inv float64
comp3_rate_percent_diff float64
comp4_rate float64
comp4_inv float64
comp4_rate_percent_diff float64
comp5_rate float64
comp5_inv float64
comp5_rate_

In [33]:
# rename date time column to month (january, february, march, april, may, june, july, august, september, october, november, december)
train_data['date_time'] = pd.to_datetime(train_data['date_time'])
train_data['date_time'] = train_data['date_time'].dt.month_name()

In [34]:
# for the columns prop_starrating and prop_review_score, a 0 represents a missing value and should be replaced by nan
train_data['prop_starrating'] = train_data['prop_starrating'].replace(0, np.nan)
train_data['prop_review_score'] = train_data['prop_review_score'].replace(0, np.nan)

In [35]:
# compute the mean of the two columns prop_location_score1 and prop_location_score2 
train_data['prop_location_score2'] = train_data['prop_location_score2'].fillna(train_data['prop_location_score1'])
train_data['prop_location_score1'] = train_data['prop_location_score1'].fillna(train_data['prop_location_score2'])
train_data['prop_location_score'] = train_data[['prop_location_score2', 'prop_location_score1']].mean(axis=1)
train_data = train_data.drop(['prop_location_score1', 'prop_location_score2'], axis=1)

In [36]:
# comb_rate mean is the mean of all not nan values in all columns that contain comb_rate
train_data['comb_rate'] = train_data[['comp1_rate', 'comp2_rate', 'comp3_rate', 'comp4_rate', 'comp5_rate', 'comp6_rate', 'comp7_rate', 'comp8_rate']].sum(axis=1, skipna=True)
train_data['comb_rate_percentage_diff'] = train_data[['comp1_rate_percent_diff', 'comp2_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate_percent_diff', 'comp5_rate_percent_diff', 'comp6_rate_percent_diff', 'comp7_rate_percent_diff', 'comp8_rate_percent_diff']].mean(axis=1, skipna=True)
train_data["comb_inv"] = train_data[['comp1_inv', 'comp2_inv', 'comp3_inv', 'comp4_inv', 'comp5_inv', 'comp6_inv', 'comp7_inv', 'comp8_inv']].sum(axis=1, skipna=True)

In [37]:
train_data = train_data.drop(['comp1_rate', 'comp2_rate', 'comp3_rate', 'comp4_rate', 'comp5_rate', 'comp6_rate', 'comp7_rate', 'comp8_rate'], axis=1)
train_data = train_data.drop(['comp1_rate_percent_diff', 'comp2_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate_percent_diff', 'comp5_rate_percent_diff', 'comp6_rate_percent_diff', 'comp7_rate_percent_diff', 'comp8_rate_percent_diff'], axis=1)
train_data = train_data.drop(['comp1_inv', 'comp2_inv', 'comp3_inv', 'comp4_inv', 'comp5_inv', 'comp6_inv', 'comp7_inv', 'comp8_inv'], axis=1)

In [38]:
# remove all rows that contain more than 50% missing values
train_data = train_data.dropna(thresh=train_data.shape[1]*0.5)

In [39]:
# remove all columns that contain more than 30% missing values
for col in train_data.columns:
    if train_data[col].isnull().sum()/train_data.shape[0] > 0.30:
        print(col, train_data[col].isnull().sum()/train_data.shape[0])
        
        train_data = train_data.drop([col], axis=1)

visitor_hist_starrating 0.949203635808466
visitor_hist_adr_usd 0.9489773507178905
srch_query_affinity_score 0.935985520981085
orig_destination_distance 0.32425766086964064
gross_bookings_usd 0.9720894886945186
comb_rate_percentage_diff 0.681246592866534


In [40]:
# handle missing values 
for col in train_data.columns:
    
    # check if numerical
    if train_data[col].dtype == 'float64' or train_data[col].dtype == 'int64':   
        train_data[col] = train_data[col].fillna(train_data[col].mean())
        
        # TODO: not always mean, sometimes median or mode
    
    # check if categorical
    if train_data[col].dtype == 'object' or train_data[col].dtype == 'bool':
        train_data[col] = train_data[col].fillna(train_data[col].value_counts().idxmax())

# check if there are still missing values
print(train_data.isnull().sum().sum())

0


In [41]:
# if not os.path.exists('plots'):
#     os.makedirs('plots')

# # plot all the categorical data
# for col in train_data.columns:
#     if train_data[col].dtype == 'object':
#         plt.figure()
#         train_data[col].value_counts().plot(kind='bar')
#         plt.title(col)
#         plt.savefig('plots/' + col + '.png')
#         plt.close()

In [42]:
# for column in train_data.columns:
    
#     # check if column is categorical
#     if train_data[column].dtype == 'object':
        
#         # check if there is a subcategory with less than 5 occurences
#         for subcategory in train_data[column].unique():
#             if train_data[train_data[column] == subcategory].shape[0] < 5:
#                 print(column, subcategory, train_data[train_data[column] == subcategory].shape[0])

In [ ]:
# TODO: remove the columns ... and ... that are not in the test set

In [43]:
# TODO: handle bins with less than 5 occurences 
# TODO: merge or to many different bins: 
# visitor_location_country_id and prop_country_id object to larger region
# site_id?
# prop_id?
# srch_destination_id?
# --> use k-means clustering to cluster these into bins with more than 5 occurences

In [44]:
# # plot each numerical feature 
# for col in train_data.columns:
#     if train_data[col].dtype == 'float64':
#         plt.figure()
#         plt.hist(train_data[col])
#         plt.title(col)
#         plt.savefig('plots/' + col + '.png')
#         plt.close()


In [45]:
# TODO transformation of features based on trend analysis 

In [46]:
# TODO: remove extreme outliers

In [ ]:
# save cleaned data in data folder
train_data.to_csv('data/train_cleaned.csv', index=False)